### Cazyからダウンロードできるファイルを使ってNCBIの配列を取得する。

In [ ]:
#このセルを実行し、Google drive に接続。アカウントを選択し、許可 を押す。
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install biopython

In [ ]:
import os
import csv
import re
from Bio import SeqIO
from Bio import Entrez

メールアドレス、ファイル名を指定する。<br>
[配列の取得にメールアドレスが必要な理由](https://qiita.com/joemphilips/items/767c67524e4b7e328834)

In [ ]:
Entrez.email = "xxx@yy.zz"#メールアドレス

#左側のフォルダマークをクリック。drive>MyDrive> から辿って、ID_list= のパスをAccession number のファイルのものに書き換える。
#パスはフォルダ名の右側に出てくる点三つのマークをクリックしてコピーすると良い。
ID_list='/content/drive/MyDrive/Colab_Notebooks/Qiita/fetch_cazy/test.txt'#拡張子を .txt にしてください

Accession numberリストの読み込み

In [ ]:

line_count = 0
id_set = set()#IDが重複しないよう set 型を使う
with open(ID_list, "r") as f:
    lines = f.readlines()
    for i in lines:
      line_count += 1
      a = i.split()#各行をスベースで区切る
      id_set.add(a[-1])#区切られた後ろから一番目の文字列を id_set に格納
print('line count:' + str(line_count))
print('id count:' + str(len(id_set)))#行数と id の数を表示

配列の取得

In [ ]:
# このセルは配列数に比例して実行に時間がかかる
z = []
# Biopython の Entrez を用いて配列を取得し、リストに格納
for i in id_set:
    handle = Entrez.efetch(db="protein", id=i, rettype="fasta", retmode="text")
    record = SeqIO.read(handle, "fasta")
    z.append(record.format("fasta"))
handle.close()

In [ ]:
#ファイルに保存する。
out = ID_list.replace('.txt', '.fasta')#入力ファイルの拡張子を「fasta」に変更して出力ファイル名とする
with open(out, 'w') as r:
    for d in z:
        r.write('%s' % d)

In [ ]:
#保存したファイルを開いて、配列の数をカウント
a = 0
with open(out, 'r') as f:
    lines = f.readlines()
    for i in lines:
        m = re.match('>', i)# > の数を数える
        if m:
            a += 1
b = '取得した配列：' + str(a) + '個'
print(b)